In [30]:
import torch
import torchvision

class MyScriptModule(torch.nn.Module):
    def __init__(self):
        super(MyScriptModule, self).__init__()
        self.means = torch.nn.Parameter(torch.tensor([103.939, 116.779, 123.68])
                                        .resize_(1, 3, 1, 1))
        self.resnet = torch.jit.trace(torchvision.models.resnet18(num_classes=10),
                                      torch.rand(1, 3, 224, 224))

    def forward(self, input):
        return self.resnet(input - self.means)

my_script_module = torch.jit.script(MyScriptModule())


In [33]:
input = torch.rand(1, 3, 100, 100)
output = my_script_module(input)
output.shape

torch.Size([1, 10])

In [80]:
@torch.jit.script
def foo(len):
    # type: (int) -> torch.Tensor
    rv = torch.zeros(3, 4)
    for i in range(len):
        if i < 10:
            rv = rv - 1.0
        else:
            rv = rv + 1.0
    return rv

print(foo.code)

def foo(len: int) -> Tensor:
  rv = torch.zeros([3, 4])
  rv0 = rv
  for i in range(len):
    if torch.lt(i, 10):
      rv1 = torch.sub(rv0, 1.)
    else:
      rv1 = torch.add(rv0, 1.)
    rv0 = rv1
  return rv0



In [81]:
print(foo.graph)

graph(%len.1 : int):
  %21 : int = prim::Constant[value=1]()
  %13 : bool = prim::Constant[value=1]() # /tmp/ipykernel_39804/1503696268.py:5:4
  %5 : NoneType = prim::Constant()
  %1 : int = prim::Constant[value=3]() # /tmp/ipykernel_39804/1503696268.py:4:21
  %2 : int = prim::Constant[value=4]() # /tmp/ipykernel_39804/1503696268.py:4:24
  %16 : int = prim::Constant[value=10]() # /tmp/ipykernel_39804/1503696268.py:6:15
  %20 : float = prim::Constant[value=1.]() # /tmp/ipykernel_39804/1503696268.py:7:22
  %4 : int[] = prim::ListConstruct(%1, %2)
  %rv.1 : Tensor = aten::zeros(%4, %5, %5, %5, %5) # /tmp/ipykernel_39804/1503696268.py:4:9
  %rv : Tensor = prim::Loop(%len.1, %13, %rv.1) # /tmp/ipykernel_39804/1503696268.py:5:4
    block0(%i.1 : int, %rv.29 : Tensor):
      %17 : bool = aten::lt(%i.1, %16) # /tmp/ipykernel_39804/1503696268.py:6:11
      %rv.27 : Tensor = prim::If(%17) # /tmp/ipykernel_39804/1503696268.py:6:8
        block0():
          %rv.5 : Tensor = aten::sub(%rv.29, %20,